##This notebook is for basic QC on sequencing data

In [ ]:
# %install_ext https://raw.githubusercontent.com/SchlossLab/ipython-mothurmagic/master/mothurmagic.py
# Only needs to be done once - gets the mothurmagic so you can run mother easily in the %notebook

In [1]:
%load_ext mothurmagic
# Loads mothurmagic so we can run mothur in the notebook using %%mothur at the top of the cell

In [2]:
%%mothur
help()

mothur > help()
For more information about a specific command type 'commandName(help)' i.e. 'read.dist(help)'

For further assistance please refer to the Mothur manual on our wiki at http://www.mothur.org/wiki, or contact Pat Schloss at mothur.bugs@gmail.com.

mothur > quit()


In [3]:
!echo $PATH
#Make sure mothur folder is in your path somehow. If not, enter 

/opt/virt_env/bin:/Users/Thea/anaconda/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/opt/X11/bin:/usr/texbin:/opt/virt_env/bin/mothur


In [ ]:
!export PATH=$PATH:/opt/virt_env/bin/mothur

First we need to cut out sequences with errors

In [4]:
! head -1000 ../../SeqData/pear_merged-2015-07-13.assembled.fastq > ../../SeqData/pear_merged-2015-07-13.assembled.mini.fastq

In [6]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-07-13.assembled.fastq \
-fastaout ../../SeqData/pear_merged_2015_07_13.assembled.demult.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering
# Note we needed to remove - dashes from the name for future mothur processing

usearch v8.0.1623_i86osx32, 4.0Gb RAM (17.2Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: tlw59@cornell.edu

01:07 1.6Mb  100.0% Filtering, 94.1% passed
   2441427  FASTQ recs (2.4M)              
    143324  Low qual recs discarded (expected errs > 1.00)
   2298103  Converted (2.3M, 94.1%)


####If the file were bigger, we would need to split it (below)

In [ ]:
!wc -l ../../SeqData/pear_merged-2015-06-30.assembled.fastq
# Counts the number of lines in the fastq file. They come in groups of 4 lines.

In [ ]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-06-30.assembled.fastq \
-fastaout ../../SeqData/pear_merged-2015-06-30.assembled.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering - the whole file is too big to do at once so needs to be split

In [ ]:
!head -16528900 ../../SeqData/pear_merged-2015-06-30.assembled.fastq > \
../../SeqData/pear_merged-2015-06-30.assembled.split1.fastq
!tail -16528900 ../../SeqData/pear_merged-2015-06-30.assembled.fastq > \
../../SeqData/pear_merged-2015-06-30.assembled.split2.fastq
# Split the data for usearch into two files, divided by the number of lines it has as determined above

In [ ]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-06-30.assembled.split1.fastq \
-fastaout ../../SeqData/pear_merged-2015-06-30.assembled.split1.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering on our split fastq file

In [ ]:
! usearch -fastq_filter ../../SeqData/pear_merged-2015-06-30.assembled.split2.fastq \
-fastaout ../../SeqData/pear_merged-2015-06-30.assembled.split2.maxee.fasta \
-fastq_maxee 1
# Using Roger Edgar's max expected error filtering on our split fastq file

In [ ]:
!cat ../../SeqData/pear_merged-2015-06-30.assembled.split1.maxee.fasta \
../../SeqData/pear_merged-2015-06-30.assembled.split2.maxee.fasta > \
../../SeqData/pear_merged_2015_06_30.assembled.maxee.fasta
# Joining the files back together into one fasta file.

###Now we examine the sequences using mothur

In [8]:
%%mothur
summary.seqs(fasta=../../SeqData/pear_merged_2015_07_13.assembled.demult.maxee.fasta, processors=4)

mothur > summary.seqs(fasta=../../SeqData/pear_merged_2015_07_13.assembled.demult.maxee.fasta, processors=4)

Using 4 processors.

Start	End	NBases	Ambigs	Polymer	NumSeqs
Minimum:	1	50	50	0	2	1
2.5%-tile:	1	187	187	0	3	57453
25%-tile:	1	294	294	0	4	574526
Median: 	1	304	304	0	5	1149052
75%-tile:	1	339	339	0	6	1723578
97.5%-tile:	1	392	392	0	9	2240651
Maximum:	1	492	492	1	82	2298103
Mean:	1	311.203	311.203	3.48113e-06	5.35651
# of Seqs:	2298103

Output File Names:
../../SeqData/pear_merged_2015_07_13.assembled.demult.maxee.summary

It took 19 secs to summarize 2298103 sequences.

mothur > quit()


Okay, we have about 2.3M sequences, and they're variable lengths, but that's sort of to be expected, because this is a variable region and we merged the reads.

In [9]:
!head ../../SeqData/pear_merged_2015_07_13.assembled.demult.maxee.fasta

>74_1
CAGTGAGTCATCGAATCTTTGCTTGCTGCATATCAATAAGCAGAGGATCTTA
>55_11
TCAGTGAATCATCGAGTCTTTGAACGCACATTGCGCCCCGTGGTATTCCGCGGGGCATGCCTGTTCGAGCGTCATTTCAA
CCCTCAAGCTCTGCTTGGTGTTGGGCCCTGCCCGCTGTGGCCGGCCCTAAAATCAGTGGCGGCGCCGTCTGGCTCTAAGC
GTAGTAATGCTCTCGCTCTAGGGTCCTGCGGTGGCTTGCCAGCAACCCCAACTTTCCATGGTTGACCTCGGATCAGGTAG
GGATACCCGCTGAACAAAAGCATATCAATAAGCGGAGGATCGA
>84_13
TTGAATCATCGAATCTTTGAACGCAAGTGGCGATGGTTTCGGCCATCATGTTTGTTTCAGTGTGTTCGGTTAATTGCATC
GTAGAATTTAATGTGACTGAAGCGATTCTTTCACTAAATACGAACTAGTGTCAGACGAAGGACATTAAGCGAACTAGTCA
